# #Install beir

In [ ]:
pip install beir


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Created wheel for beir: filename=beir-2.0.0-py3-none-any.whl size=63549 sha256=53133954fab1bb7095f722b2086c3df4e4d69c5b21bbb1

# #Installing Required Libraries for NLP and Search-Based Tasks

In [ ]:
pip install torch transformers sentence-transformers beir faiss-cpu datasets sklearn


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


# #Downloading and Unzipping BEIR Dataset for Benchmarking

In [ ]:
from beir import util

dataset_name = "nq"  # Choosing from nq, hotpotqa, fiqa
url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{dataset_name}.zip"
out_dir = f"./datasets/{dataset_name}"
data_path = util.download_and_unzip(url, out_dir)


./datasets/nq/nq.zip:   0%|          | 0.00/475M [00:00<?, ?iB/s]

# #Tokenizing Query and Passage Using Pre-trained Tokenizer

In [ ]:
from transformers import AutoTokenizer

# Defining query and passage
query = "What is the capital of France?"  # Example query
passage = "Paris is the capital and most populous city of France."  # Example passage

# Loading the tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Tokenizing the query and passage
query_tokens = tokenizer(query, return_tensors='pt', max_length=512, truncation=True)
passage_tokens = tokenizer(passage, return_tensors='pt', max_length=512, truncation=True)

print(query_tokens)
print(passage_tokens)


{'input_ids': tensor([[ 101, 2054, 2003, 1996, 3007, 1997, 2605, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[  101,  3000,  2003,  1996,  3007,  1998,  2087, 20151,  2103,  1997,
          2605,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# #Generating Embeddings for Query and Passages Using SentenceTransformer

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
query_embedding = model.encode(query, convert_to_tensor=True)
passage_embeddings = model.encode(passages, convert_to_tensor=True)


In [ ]:
print("Passages:", passages)  # Checking if passages list is populated


Passages: ['Passage 1 text here.', 'Passage 2 text here.', 'Passage 3 text here.']


# #FAISS-based Passage Retrieval Using SentenceEmbeddings

In [ ]:
import faiss
from sentence_transformers import SentenceTransformer


passages = [
    'Passage 1 text here.',
    'Passage 2 text here.',
    'Passage 3 text here.'
]

# Printing the passages to verify they are populated
print("Passages:", passages)

model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the passages
passage_embeddings = model.encode(passages, convert_to_tensor=True)


print("passage_embeddings shape: ", passage_embeddings.shape)

if len(query_embedding.shape) == 1:
    query_embedding = query_embedding.unsqueeze(0)

if len(passage_embeddings.shape) == 1:
    passage_embeddings = passage_embeddings.unsqueeze(0)

# Initializing FAISS index using the correct embedding dimension (from query_embedding)
index = faiss.IndexFlatL2(query_embedding.shape[1])

index.add(passage_embeddings.cpu().numpy())

assert len(passage_embeddings) == len(passages), "Mismatch between passages and embeddings"

# Perform FAISS search to retrieve top k passages
k = 10
_, retrieved_indices = index.search(query_embedding.cpu().numpy(), k)

retrieved_passages = [passages[i] for i in retrieved_indices[0]]

print("Retrieved Passages: ", retrieved_passages)


Passages: ['Passage 1 text here.', 'Passage 2 text here.', 'Passage 3 text here.']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


passage_embeddings shape:  torch.Size([3, 384])
Retrieved Passages:  ['Passage 2 text here.', 'Passage 1 text here.', 'Passage 3 text here.', 'Passage 3 text here.', 'Passage 3 text here.', 'Passage 3 text here.', 'Passage 3 text here.', 'Passage 3 text here.', 'Passage 3 text here.', 'Passage 3 text here.']


# #Re-ranking Retrieved Passages Using Cross-Encoder

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

cross_encoder = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-MiniLM-L-12-v2")
tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L-12-v2")

inputs = tokenizer([query] * len(retrieved_passages), retrieved_passages, return_tensors='pt', padding=True)
scores = cross_encoder(**inputs).logits
sorted_indices = scores.squeeze().argsort(descending=True)
reranked_passages = [retrieved_passages[i] for i in sorted_indices]


# #Calculating NDCG@10 for Query-Passage Ranking

In [ ]:
from sklearn.metrics import ndcg_score

# Example: true relevance scores (e.g., ground truth relevance for query-passage pairs)
true_relevances = [3, 2, 1, 0]  # Ground truth relevance scores for 4 passages

# Example: predicted relevance scores (e.g., model predicted scores for the same passages)
predicted_relevances = [0.9, 0.8, 0.6, 0.3]  # Predicted scores (similarity values)

# Compute NDCG@10 score
ndcg = ndcg_score([true_relevances], [predicted_relevances], k=10)

print(f"NDCG@10: {ndcg}")


NDCG@10: 1.0
